[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pantelis-nlp/tutorial-nlp-notebooks/blob/main/rnn_language_model.ipynb)

# Simple RNN Language Model

In [ ]:
data = 'Chios island is crescent or kidney shaped, 50 km (31 mi) long from north to south, and 29 km (18 mi) at its widest, covering an area of 842.289 km2 (325.210 sq mi).[2] The terrain is mountainous and arid, with a ridge of mountains running the length of the island. The two largest of these mountains, Pelineon (1,297 m (4,255 ft)) and Epos (1,188 m (3,898 ft)), are situated in the north of the island. The center of the island is divided between east and west by a range of smaller peaks, known as Provatas.'
# data I/O
# data = open('input.txt', 'r').read() # should be simple plain text file - you can use any (small) file in txt format from the web or type your own. 

In [ ]:
import numpy as np

In [ ]:
# creating a vocabulary of unique characters
chars = list(set(data))                                                   
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 508 characters, 43 unique.


In [ ]:
# Data pre-processing
# creating a dictionary, mapping characters to index and index to characters
char_to_ix = { ch:i for i,ch in enumerate(chars) }
print(char_to_ix)
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(ix_to_char)

{'9': 0, 'd': 1, 's': 2, '0': 3, 'c': 4, '7': 5, 'a': 6, 'u': 7, 'P': 8, 'b': 9, 'n': 10, ')': 11, 'k': 12, 'g': 13, 'o': 14, 'r': 15, 'l': 16, '.': 17, 'p': 18, '2': 19, '1': 20, '[': 21, '5': 22, 'i': 23, 'w': 24, 'q': 25, 'm': 26, '(': 27, 't': 28, '3': 29, '4': 30, ']': 31, 'v': 32, ',': 33, 'C': 34, 'y': 35, '8': 36, 'h': 37, 'T': 38, 'f': 39, 'E': 40, 'e': 41, ' ': 42}
{0: '9', 1: 'd', 2: 's', 3: '0', 4: 'c', 5: '7', 6: 'a', 7: 'u', 8: 'P', 9: 'b', 10: 'n', 11: ')', 12: 'k', 13: 'g', 14: 'o', 15: 'r', 16: 'l', 17: '.', 18: 'p', 19: '2', 20: '1', 21: '[', 22: '5', 23: 'i', 24: 'w', 25: 'q', 26: 'm', 27: '(', 28: 't', 29: '3', 30: '4', 31: ']', 32: 'v', 33: ',', 34: 'C', 35: 'y', 36: '8', 37: 'h', 38: 'T', 39: 'f', 40: 'E', 41: 'e', 42: ' '}


Our aim is to predict the next character given a set of previous characters from our data string.

For our RNN and CNN implementations, we would take a sequence of length 25 characters as inputs to predict the next character.

## RNN

In [ ]:
# see here for notation http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture10.pdf
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""

'\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nBSD License\n'

<img src='https://drive.google.com/uc?id=12ha59vACcd8eCEPAQdbZ4-axPGPKnn7F' width="700">


**Inputs to RNN**
- $x_1$ to $x_{25}$ is the input sequence of 25 characters, one character given as input to RNN at each time step


**Hidden state of RNN**
- The state consists of a single 'hidden' vector h
- At every time step, a recurrence function $f_W$ with parameters $W_{xh}$, $W_{hh}$ and $b_h$ is applied to the input $x_t$ and the output from the previous hidden state $h_{t-1}$, to generate $h_t$

$ \qquad \qquad \qquad \qquad h_t = f_W (h_{t-1},x_t)$

$ \qquad \qquad \qquad \qquad \; \; \; \; = tanh (W_{hh}h_{t-1} + W_{xh}x_t + b_h)$

**Outputs of the RNN**
- $y_T$ is the character that our network would predict after $T=25$ time steps
- At each time step, a $y_t$ is calculated as

$ \qquad \qquad \qquad \qquad y_t = W_{hy}h_t + b_y$

- The softmax of $y_t$ is the set of probabilities of occurance of each unique character in the input data

$ \qquad \qquad \qquad \qquad softmax(y_t)$

- At each time step, from $t=1$ to $25$, loss is calculated from the set of predicted probabilities. 

$ \qquad \qquad \qquad \qquad L_t = Cross\;Entropy(softmax(y_t), target_t)$

$ \qquad $ where the $target$ is the next character to the input sequence in the data string

- The total loss is the sum of all the losses from the previously unrolled steps

$ \qquad \qquad \qquad \qquad L = ∑_{t=0}^{24}L_t$

All the weights $W_{xh}$, $W_{hh}$, $b_h$, $W_{hy}$ and $b_y$ are reused at each time step.


**Hyperparameters**

- the size of hidden state of neurons
- the sequence length or the time steps to unroll, which is 25 in our case
- optimizer we use here is Adagrad
- the learning rate for Adagrad optimizer

In [ ]:
# hyperparameters
hidden_size = 100           # size of hidden layer of neurons
seq_length = 25             # number of time steps to unroll the RNN for, taking 25 previous characters to predict the next
learning_rate = 1e-1

**Dimensions of tensors**

Input:
- Each character from the data string is pre-processed before being fed then into the RNN
- From each sequence of 25 characters (for 25 time-steps) from the data string, we create an 'inputs' list of tokenized integer values 
- Each character is converted to an integer token index using 'char_to_ix' function, which maps each character to a number between 0 and 42 (as there are 43 unique characters in our data)
- The integer tokens from the 'inputs' list are then one-hot encoded in 1-of-k representations, ie, into vectors of size 43 (k=43 unique characters in our data), which are fed as inputs to the RNN

$ \qquad $ => dimension of input $x_t =$ (43,1)

Targets:
- For each input in the 'inputs' list, we create a 'target' list consisting of the subsequent character's integer token
- Our 'targets' list, which is used during the cross-entropy loss calculation, is of length 25

Predicted output:
- The predicted outputs are the probabilities of the next characters
- Since k=43 unique characters, the unnormalized log probabilities for next chars is given by $y_t =$

$ \qquad $ => dimension of output $y_t =$ (43,1)

- The $softmax(y_t)$ gives the class probabilities for next characters

- The probabilities are then converted into one-hot encoded vectors using argmax

- The one-hot encoded vectors are converted into integer tokens and then to a single character using 'ix_to_char' function

Hidden layers:
- Since we have chosen 100 neurons in the hidden layer,

$ \qquad $ => dimension of hidden state $h_t =$ (100,1)

Model parameters:
- Given the hidden_size=100,  input x dimension=(43,1) and output y dimension=(43,1):

$ \qquad $ => dimension of $W_{xh} =$ (100,43), 

$ \qquad $ => dimension of $W_{hh} =$ (100,100), 

$ \qquad $ => dimension of $b_{h} =$ (100,1), 

$ \qquad $ => dimension of $W_{hy} =$ (43,100), 

$ \qquad $ => dimension of $b_{y} =$ (43,1)

In [ ]:
# model parameters
# we set the initial values of the weights randomly from a normal distribution and set all the bias to zero

Wxh = np.random.randn(hidden_size, vocab_size)*0.01   # input to hidden, shape = (hidden_size, vocab_size) = (100,43)
Whh = np.random.randn(hidden_size, hidden_size)*0.01  # hidden to hidden, shape = (hidden_size, hidden_size) = (100,100)
Why = np.random.randn(vocab_size, hidden_size)*0.01   # hidden to output, shape = (vocab_size, hidden_size) = (43,100)
bh = np.zeros((hidden_size, 1))                       # hidden bias, shape  = (hidden_size, 1) = (100,1)
by = np.zeros((vocab_size, 1))                        # output bias, shape  = (vocab_size, 1) = (43,1)

**Forward Pass**

- Forward through entire sequence $x_1$ to $x_{25}$ to compute loss

- Calculate hidden states at each time step

$ \qquad \qquad \qquad \qquad h_t = tanh (W_{hh}h_{t-1} + W_{xh}x_t + b_h)$

- Calculate output $y_t$ 

$ \qquad \qquad \qquad \qquad y_t = W_{hy}h_t + b_y$

- The softmax of $y_t$ is the set of probabilities of occurance of each unique character in the input data

$ \qquad \qquad \qquad \qquad p_t = softmax(y_t)$

- Calculate loss at each time step 

$ \qquad \qquad \qquad \qquad L_t = Cross\;Entropy(p_t, target_t)$

- Calculate the total loss, which is the negative log likelihood of our model

$ \qquad \qquad \qquad \qquad L = ∑_{t=0}^{24}L_t$

$ \qquad \qquad \qquad \qquad \; \; \; \; = - ∑_t log \; p_{model} (y_t | x_1,...,x_t)$

**Backpropogation Through Time**

- Backward through entire sequence to compute gradient
- The nodes include parameters $W_{xh}$, $W_{hh}$, $b_h$, $W_{hy}$ and $b_y$ 
- The inputs and outputs of nodes are $x_t$, $h_t$, $y_t$, $p_t$ and $L_t$ at time-step $t$
- We'll use the suffix $(i)$ to indicate the $i^{th}$ sample

*Gradients on the internal nodes:*

- We'll be computing the gradients recursively starting with the nodes immediately preceding the final loss

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial L_t}  = 1 $

- The gradient with respect to the softmax layer would be:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial y_{(i)t}}  = p_{(i)t} -1 $

- At t=25, the gradient with respect to the hidden layer would be:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial h_{t=25}}  = {W_{hy}}^T \frac{\partial L}{\partial y_{t=25}} $

- We can now iterate backwards from t=24 down to t=1:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial h_{t}}  =  \bigg(\frac{\partial h_{t+1}}{\partial h_{t}}\bigg)^T \frac{\partial L}{\partial h_{t+1}} + \bigg(\frac{\partial y_{t}}{\partial h_{t}}\bigg)^T \frac{\partial L}{\partial y_{t}}  $

$ \qquad \qquad \qquad \qquad \qquad = (W_{hh})^T \; diag\bigg(1-(h_{t+1})^2\bigg) \frac{\partial L}{\partial h_{t+1}}  + (W_{hy})^T \frac{\partial L}{\partial y_{t}} $

$ \qquad \qquad $ where $diag\bigg(1-(h_{t+1})^2\bigg)$ indicates the diagonal matrix containing the elements $1-(h_{(i)t+1})^2$

*Gradients on the parameter nodes:*

- Gradients with respect to $W_{hy}$:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial W_{hy}} = \sum_t  \sum_i \frac{\partial L}{\partial y_{(i)t}} \frac{\partial y_{(i)t}}{\partial W_{hy}} $

$ \qquad \qquad \qquad \qquad  \qquad = \sum_t  \frac{\partial L}{\partial y_t} (h_t)^T$

- Gradients with respect to $b_y$:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial b_{y}} = \sum_t \bigg(\frac{\partial y_t}{\partial b_{y}}\bigg)^T \frac{\partial L}{\partial y_{t}} $

$ \qquad \qquad \qquad \qquad  \qquad = \sum_t \frac{\partial L}{\partial y_{t}} $

- Gradients with respect to $W_{hh}$:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial W_{hh}} = \sum_t  \sum_i \frac{\partial L}{\partial h_{(i)t}} \frac{\partial h_{(i)t}}{\partial W_{hh}} $

$ \qquad \qquad \qquad \qquad  \qquad = \sum_t diag\bigg(1-(h_{t})^2\bigg)  \frac{\partial L}{\partial h_t} (h_{t-1})^T$

- Gradients with respect to $b_h$:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial b_{h}} = \sum_t \bigg(\frac{\partial h_t}{\partial b_{h}}\bigg)^T \frac{\partial L}{\partial h_{t}} $

$ \qquad \qquad \qquad \qquad  \qquad = \sum_t diag\bigg(1-(h_{t})^2\bigg)  \frac{\partial L}{\partial h_{t}} $

- Gradients with respect to $W_{xh}$:

$ \qquad \qquad \qquad \qquad \frac{\partial L}{\partial W_{xh}} = \sum_t  \sum_i \frac{\partial L}{\partial h_{(i)t}} \frac{\partial h_{(i)t}}{\partial W_{xh}} $

$ \qquad \qquad \qquad \qquad  \qquad = \sum_t diag\bigg(1-(h_{t})^2\bigg)  \frac{\partial L}{\partial h_t} (x_t)^T$

In [ ]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  perform forward and backward pass
  returns the loss, gradients on model parameters, and last hidden state
  """

  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0

  # forward pass: compute loss going forward
  for t in range(len(inputs)):                                         # looping for t timesteps, which is the size of the length of inputs
    xs[t] = np.zeros((vocab_size,1))                                   # xs = one-hot encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)    # hs_t = tanh(W_hh.hs_t-1 + W_xh.xs_t + b_h) -> hidden state
    ys[t] = np.dot(Why, hs[t]) + by                                    # ys = W_hy.hs_t + b_y -> unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))                      # ps = softmax(ys) -> probabilities for next chars
    loss += -np.log(ps[t][targets[t],0])                               # cross-entropy loss

  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)      # create numpy arrays for right size for the weights
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)                                    # create numpy arrays for right size for the biasses
  dhnext = np.zeros_like(hs[0])                                                      # h_t-1 for the first iteration is set to all zeros
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1                      # backprop into y by taking gradient for softmax (http://cs231n.github.io/neural-networks-case-study/#grad)
    dWhy += np.dot(dy, hs[t].T)              # gradient for Why
    dby += dy                                # gradient for by
    dh = np.dot(Why.T, dy) + dhnext          # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh         # backprop through tanh nonlinearity
    dbh += dhraw                             # gradient for bh
    dWxh += np.dot(dhraw, xs[t].T)           # gradient for Wxh
    dWhh += np.dot(dhraw, hs[t-1].T)         # gradient for Whh
    dhnext = np.dot(Whh.T, dhraw)            # calculate h_t-1 for the next iteration
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam)                              # clip gradients to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [ ]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  predicts probabilities for each character 
  returns the set of predicted indices with the highest probabilities
  """
  # at test-time sample characters one at a time, feed back to model for next character prediction
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1                                              # x = one-hot encode the input for seed_ix letter in 1-of-k representation
  ixes = []
  for t in range(n):
    # predicting the next character
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)         # h_t = tanh(W_hh.h_t-1 + W_xh.x_t + b_h) -> hidden state                    
    y = np.dot(Why, h) + by                                   # y = W_hy.h_t + b_y -> unnormalized log probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))                         # p = softmax(y) -> probabilities for next chars
    ix = np.random.choice(range(vocab_size), p=p.ravel())     # p.ravel gives the probabilities of each entry, with the maximum ix at argmax
    x = np.zeros((vocab_size, 1))  
    x[ix] = 1                                                 # convert probabilities to one-hot encoded vectors in 1-of-k representation
    ixes.append(ix)
  return ixes                                                 # return all the indices to convert them into characters and print the predictions

In [ ]:
# p-data pointer, n-iteration counter
n, p = 0, 0                        # setting both to zero in the beginning

# memory variables for Adagrad, initialized to all zeros
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)    
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 

# loss at time instance 0
smooth_loss = -np.log(1.0/vocab_size)*seq_length 

# while True:
# running for 80000 epochs
for i in range(80000):

  # Data pre-processing to prepare inputs and targets
  if p+seq_length+1 >= len(data) or n == 0:                        # sweeping from left to right in steps seq_length=25 long
    hprev = np.zeros((hidden_size,1))                              # reset RNN memory
    p = 0                                                          # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]         # inputs are tokens each of length seq_length=25
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]    # targets are the tokens of the subsequent characters for each input sequence

  # Model testing
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)                   # sample from the model and predict characters every 1000 iterations
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)           # convert tokens into characters and add it to the list of previous predictions
    print('----\n %s \n----' % (txt, ))                         # print model predictions

  # Model training
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)   # forward seq_length characters through the net and fetch gradient
  smooth_loss = smooth_loss * 0.999 + loss * 0.001                            # RNN adds all the losses from the previously unrolled steps
  if n % 1000 == 0: print('iter %d, loss: %f' % (n, smooth_loss))             # print progress
  
  # parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)              # adagrad parameter update

  p += seq_length                                                       # move data pointer
  n += 1                                                                # iteration counter 

----
 oq7wTvb.bksek.CTolkw]dl31[esuqwsww4gq2Ctd,)7P,fi00is3,Thi]2oEhtvoP.4Cybf83go90pa3y0PEs1req]v]khaeggivCf[8P5Cqpb[E5smtq)([gT0C5ns)sa4hTekk42yd]wqndTlk8u9m41bs,f078ew]7T1(2]T(,wp]P9)2flyybb.8hTPuukkb4]. 
----
iter 0, loss: 94.030000
----
 hiosh inseinoe o) s, kmt)e)algeidf  iolialsarrand. (h8 b39 kthe oris knoPires sa naf ates s47 f mitwf (4T2 a ndind awretf of th th2 i) apesotwed mt)  sniundemees thib  the ind is9inos1. Ped and t)ed i 
----
iter 1000, loss: 72.848263
----
 liss (Pi) ind lrst.[210 wnountaind we gestwt, islan a ge nten longet, is lanler t aid indwrmownwind islate in the nseanded ast)) kn5 s (31 mountast ipnd and islant orerineinescencer, aateain afntereth 
----
iter 2000, loss: 41.491765
----
 heos inge of these mountwre teon (1, Th the a st)), [1 and west bed west 4,188 ft)), an isis and Epos (1,255 ak iountanenty mowntan (1,297 m (4,18, st.[2] The the island. The center in the of the isla 
----
iter 3000, loss: 21.440355
----
 hios island is aped, 508 mounn(31 mi)

In [ ]:
from numpy import array
import keras as K
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout

In [ ]:
# split input sequence into inputs X of size = n_steps and targets Y with the subsequent integer after each X
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps                                     # find the end of this pattern
		if end_ix > len(sequence)-1:                             # condition to check if we are beyond the input sequence
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]      # gather input and output parts of the pattern
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# creating a vocabulary of unique characters
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

# creating a dictionary, mapping characters to index and index to characters
char_to_ix = { ch:i for i,ch in enumerate(chars) }
print(char_to_ix)
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(ix_to_char)

data has 508 characters, 43 unique.
{'9': 0, 'd': 1, 's': 2, '0': 3, 'c': 4, '7': 5, 'a': 6, 'u': 7, 'P': 8, 'b': 9, 'n': 10, ')': 11, 'k': 12, 'g': 13, 'o': 14, 'r': 15, 'l': 16, '.': 17, 'p': 18, '2': 19, '1': 20, '[': 21, '5': 22, 'i': 23, 'w': 24, 'q': 25, 'm': 26, '(': 27, 't': 28, '3': 29, '4': 30, ']': 31, 'v': 32, ',': 33, 'C': 34, 'y': 35, '8': 36, 'h': 37, 'T': 38, 'f': 39, 'E': 40, 'e': 41, ' ': 42}
{0: '9', 1: 'd', 2: 's', 3: '0', 4: 'c', 5: '7', 6: 'a', 7: 'u', 8: 'P', 9: 'b', 10: 'n', 11: ')', 12: 'k', 13: 'g', 14: 'o', 15: 'r', 16: 'l', 17: '.', 18: 'p', 19: '2', 20: '1', 21: '[', 22: '5', 23: 'i', 24: 'w', 25: 'q', 26: 'm', 27: '(', 28: 't', 29: '3', 30: '4', 31: ']', 32: 'v', 33: ',', 34: 'C', 35: 'y', 36: '8', 37: 'h', 38: 'T', 39: 'f', 40: 'E', 41: 'e', 42: ' '}


In [ ]:
seq_length = 25        # number of characters in each input sequence from which the next char is predicted

In [ ]:
data_tokens = [char_to_ix[ch] for ch in list(data)]               # convert all characters in data string into t tokens
X_tokens, y_tokens = split_sequence(data_tokens, seq_length)      # split into samples, inputs X and targets y
for i in range(len(X_tokens)):   
	print(X_tokens[i], y_tokens[i])                                 # summarize the split input and target tokens

[34 37 23 14  2 42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28
 42] 14
[37 23 14  2 42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42
 14] 15
[23 14  2 42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14
 15] 42
[14  2 42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15
 42] 12
[ 2 42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42
 12] 23
[42 23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12
 23] 1
[23  2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23
  1] 10
[ 2 16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23  1
 10] 41
[16  6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23  1 10
 41] 35
[ 6 10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23  1 10 41
 35] 42
[10  1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23  1 10 41 35
 42] 2
[ 1 42 23  2 42  4 15 41  2  4 41 10 28 42 14 15 42 12 23  1 10 41 35 42
  2] 37
[42 23  2 42  4 15 41  2  4 41

In [ ]:
num_samples = X_tokens.shape[0]

In [ ]:
print(X_tokens.shape)     # shape of tokenized input array with sequence length = 25
print(y_tokens.shape)     # shape of tokenized target array
print(num_samples)        # total number of inputs to the CNN
print(seq_length)         # number of characters in each input sequence from which the next char is predicted
print(vocab_size)         # total number of unique characters, ie, number of output classes

(483, 25)
(483,)
483
25
43


In [ ]:
# function to one-hot encode the data for each unique class k=vocab_size
def onehot_encoding(data):
  if (len(data.shape)==2):
    onehot_data = np.zeros((data.shape[0] * data.shape[1], data.max()+1), dtype=int)   # a 3D array of 0's
    onehot_data[np.arange(data.shape[0] * data.shape[1]), data.flatten()] = 1          # replace 0 with 1 at that index of the original array
    onehot_data = onehot_data.reshape(data.shape[0], data.shape[1], vocab_size)        # reshape into [num_samples, seq_length, vocab_size] format
  elif (len(data.shape)==1):
    onehot_data = np.zeros((data.shape[0], data.max()+1), dtype=int)                   # a 2D array of 0's
    onehot_data[np.arange(data.shape[0]), data.flatten()] = 1                          # replace 0 with 1 at that index of the original array
    onehot_data = onehot_data.reshape(data.shape[0], vocab_size)                       # reshape into [num_samples, vocab_size] format
  return onehot_data

In [ ]:
X_onehot = onehot_encoding(X_tokens)  # one-hot encode the inputs
X_onehot.shape                        # shape = [num_samples, seq_length, vocab_size]

(483, 25, 43)

In [ ]:
y_onehot = onehot_encoding(y_tokens)  # one-hot encode the targets
y_onehot.shape                        # shape = [num_samples, vocab_size]

(483, 43)

In [ ]:
# reset model
tf.keras.backend.clear_session

# define model
model = Sequential()

# conv1d layer with input dim (seq_length, vocab_size), ReLU activation, 64 7x7 filters
model.add(Conv1D(filters=64, kernel_size=7, activation='relu', input_shape=(seq_length, vocab_size)))  

# maxpool layer, with pool size=2
model.add(MaxPooling1D(pool_size=2))

# flatten the data before passing it to a fully connected / dense layer
model.add(Flatten())

# a dense layer with output size=43 for the 43 unique classes and a softmax activation to give the 43 class probabilities
model.add(Dense(43, activation='softmax'))

# compile the model with adam optimizer and cross entropy loss. Print accuracy metrics while training the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 19, 64)            19328     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense (Dense)               (None, 43)                24811     
                                                                 
Total params: 44,139
Trainable params: 44,139
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_input = X_onehot
y_target = y_onehot

In [ ]:
# function to test the model and predict characters after each epoch
class PredictionCallback(tf.keras.callbacks.Callback):    
  def on_epoch_end(self, epoch, logs={}):
    txt = ''
   
    x = X_input[0:1,:,:]                          # set the first input as the first 25 characters
    for i in range(seq_length):                   # convert the one_hot encoded x inputs to tokens and then to characters, add them to string 'txt'
      txt = ''.join([txt,ix_to_char[np.argmax(x[0,i,:])]])

    n = 0                                         # data pointer
    while(n < num_samples):                       # run the prediction loop until the end of total number of inputs to the CNN
      n = n + 1            
      y_pred = self.model.predict(x, verbose=0)   # predict the class probabilities for the next character from a sequence of 25 encoded characters
      ix = np.argmax(y_pred)                      # find the index of the value with the largest probability
      txt = ''.join([txt,ix_to_char[ix]])         # convert the prediction into character and add it to the list of previous predictions 

      # at test-time, feed back the predicted character to model for next character prediction
      ypred_onehot = np.zeros((1,vocab_size))     # one-hot encode the predicted probabilities
      ypred_onehot[:,ix] = 1

      x = x.reshape(seq_length,vocab_size)        # reshape x into [seq_length, vocab_size] before stacking y_pred
      x = np.vstack([x[1:,:],ypred_onehot])       # remove the first character from x and stack y_pred for the next iteration
      x = x.astype(int)                           # numpy vstack returns a float array, convert it into an integer array
      x = x.reshape(1,seq_length,vocab_size)      # reshape the input x into [1, seq_length, vocab_size]

    print('----\n %s \n----' % (txt, ))           # print the entire string of our model predictions

In [ ]:
# train the model for 100 epochs on X_input
# call the PredictionCallback() function to predict all characters and print them after each epoch
# print the loss and accuracies after each epoch
hist = model.fit(X_input, y_target, epochs=100, verbose=2, callbacks=[PredictionCallback()])

Epoch 1/100
----
 Chios island is crescent                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
----
16/16 - 21s - loss: 3.5256 - accuracy: 0.1304 - 21s/epoch - 1s/step
Epoch 2/100
----
 Chios island is crescent                                                                                                                                                                                                                                                                                                                                                                     